# Object Detection in an Urban Environment

## Data

For this project, we will be using data from the [Waymo Open dataset](https://waymo.com/open/).

[OPTIONAL] - The files can be downloaded directly from the website as tar files or from the [Google Cloud Bucket](https://console.cloud.google.com/storage/browser/waymo_open_dataset_v_1_2_0_individual_files/) as individual tf records. We have already provided the data required to finish this project in the workspace, so you don't need to download it separately.

## Structure

### Data

The data you will use for training, validation and testing is organized as follow:
```
/home/workspace/data/waymo
    - training_and_validation - contains 97 files to train and validate your models
    - train: contain the train data (empty to start)
    - val: contain the val data (empty to start)
    - test - contains 3 files to test your model and create inference videos
```

The `training_and_validation` folder contains file that have been downsampled: we have selected one every 10 frames from 10 fps videos. The `testing` folder contains frames from the 10 fps video without downsampling.

You will split this `training_and_validation` data into `train`, and `val` sets by completing and executing the `create_splits.py` file.

### Experiments
The experiments folder will be organized as follow:
```
experiments/
    - pretrained_model/
    - exporter_main_v2.py - to create an inference model
    - model_main_tf2.py - to launch training
    - reference/ - reference training with the unchanged config file
    - experiment0/ - create a new folder for each experiment you run
    - experiment1/ - create a new folder for each experiment you run
    - experiment2/ - create a new folder for each experiment you run
    - label_map.pbtxt
    ...
```

## Prerequisites

### Local Setup

For local setup if you have your own Nvidia GPU, you can use the provided Dockerfile and requirements in the [build directory](./build).

Follow [the README therein](./build/README.md) to create a docker container and install all prerequisites.

### Download and process the data

**Note:** ”If you are using the classroom workspace, we have already completed the steps in the section for you. You can find the downloaded and processed files within the `/home/workspace/data/preprocessed_data/` directory. Check this out then proceed to the **Exploratory Data Analysis** part.

The first goal of this project is to download the data from the Waymo's Google Cloud bucket to your local machine. For this project, we only need a subset of the data provided (for example, we do not need to use the Lidar data). Therefore, we are going to download and trim immediately each file. In `download_process.py`, you can view the `create_tf_example` function, which will perform this processing. This function takes the components of a Waymo Tf record and saves them in the Tf Object Detection api format. An example of such function is described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). We are already providing the `label_map.pbtxt` file.

You can run the script using the following command:
```
python download_process.py --data_dir {processed_file_location} --size {number of files you want to download}
```

You are downloading 100 files (unless you changed the `size` parameter) so be patient! Once the script is done, you can look inside your `data_dir` folder to see if the files have been downloaded and processed correctly.

### Classroom Workspace

In the classroom workspace, every library and package should already be installed in your environment. You will NOT need to make use of `gcloud` to download the images.

## Instructions

### Exploratory Data Analysis

You should use the data already present in `/home/workspace/data/waymo` directory to explore the dataset! This is the most important task of any machine learning project. To do so, open the `Exploratory Data Analysis` notebook. In this notebook, your first task will be to implement a `display_instances` function to display images and annotations using `matplotlib`. This should be very similar to the function you created during the course. Once you are done, feel free to spend more time exploring the data and report your findings. Report anything relevant about the dataset in the writeup.

Keep in mind that you should refer to this analysis to create the different spits (training, testing and validation).


### Create the training - validation splits
In the class, we talked about cross-validation and the importance of creating meaningful training and validation splits. For this project, you will have to create your own training and validation sets using the files located in `/home/workspace/data/waymo`. The `split` function in the `create_splits.py` file does the following:
* create three subfolders: `/home/workspace/data/train/`, `/home/workspace/data/val/`, and `/home/workspace/data/test/`
* split the tf records files between these three folders by symbolically linking the files from `/home/workspace/data/waymo/` to `/home/workspace/data/train/`, `/home/workspace/data/val/`, and `/home/workspace/data/test/`

Use the following command to run the script once your function is implemented:
```
python create_splits.py --data-dir /home/workspace/data
```

### Edit the config file

Now you are ready for training. As we explain during the course, the Tf Object Detection API relies on **config files**. The config that we will use for this project is `pipeline.config`, which is the config for a SSD Resnet 50 640x640 model. You can learn more about the Single Shot Detector [here](https://arxiv.org/pdf/1512.02325.pdf).

First, let's download the [pretrained model](http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz) and move it to `/home/workspace/experiments/pretrained_model/`.

We need to edit the config files to change the location of the training and validation files, as well as the location of the label_map file, pretrained weights. We also need to adjust the batch size. To do so, run the following:
```
python edit_config.py --train_dir /home/workspace/data/train/ --eval_dir /home/workspace/data/val/ --batch_size 2 --checkpoint /home/workspace/experiments/pretrained_model/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0 --label_map /home/workspace/experiments/label_map.pbtxt
```
A new config file has been created, `pipeline_new.config`.

### Training

You will now launch your very first experiment with the Tensorflow object detection API. Move the `pipeline_new.config` to the `/home/workspace/experiments/reference` folder. Now launch the training process:
* a training process:
```
python experiments/model_main_tf2.py --model_dir=experiments/reference/ --pipeline_config_path=experiments/reference/pipeline_new.config
```
Once the training is finished, launch the evaluation process:
* an evaluation process:
```
python experiments/model_main_tf2.py --model_dir=experiments/reference/ --pipeline_config_path=experiments/reference/pipeline_new.config --checkpoint_dir=experiments/reference/
```

**Note**: Both processes will display some Tensorflow warnings, which can be ignored. You may have to kill the evaluation script manually using
`CTRL+C`.

https://knowledge.udacity.com/questions/734193
**Note from Ray: try change the evaluation frequency**
You don't need to wait for full 25k steps. To evaluate the performance, you can start another new process thus it can evlauted when training. The new process will wait for the result from training process and evalue it, for example, each 1k steps



To monitor the training, you can launch a tensorboard instance by running `python -m tensorboard.main --logdir experiments/reference/`. You will report your findings in the writeup.

### Improve the performances

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the config file to improve this model https://github.com/tensorflow/models/tree/master/research/object_detection/protos. 


One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. To help you visualize these augmentations, we are providing a notebook: `Explore augmentations.ipynb`. Using this notebook, try different data augmentation combinations and select the one you think is optimal for our dataset. Justify your choices in the writeup.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer https://github.com/tensorflow/models/blob/master/research/object_detection/protos/optimizer.proto, learning rate, scheduler etc

* experiment with the architecture. The Tf Object Detection API [model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) offers many architectures. Keep in mind that the `pipeline.config` file is unique for each architecture and you will have to edit it.

**Important:** If you are working on the workspace, your storage is limited. You may to delete the checkpoints files after each experiment. You should however keep the `tf.events` files located in the `train` and `eval` folder of your experiments. You can also keep the `saved_model` folder to create your videos.


### Creating an animation
#### Export the trained model
Modify the arguments of the following function to adjust it to your models:

```
python experiments/exporter_main_v2.py --input_type image_tensor --pipeline_config_path experiments/reference/pipeline_new.config --trained_checkpoint_dir experiments/reference/ --output_directory experiments/reference/exported/
```

This should create a new folder `experiments/reference/exported/saved_model`. You can read more about the Tensorflow SavedModel format [here](https://www.tensorflow.org/guide/saved_model).

Finally, you can create a video of your model's inferences for any tf record file. To do so, run the following command (modify it to your files):
```
python inference_video.py --labelmap_path label_map.pbtxt --model_path experiments/reference/exported/saved_model --tf_record_path /data/waymo/testing/segment-12200383401366682847_2552_140_2572_140_with_camera_labels.tfrecord --config_path experiments/reference/pipeline_new.config --output_path animation.gif
```


https://knowledge.udacity.com/questions/751195

I fixed it by changing the extension from .gif to .mp4





### GPU Monitor
nvidia-smi


### If GPU is not enabled, to enable GPU in Workspace
https://knowledge.udacity.com/questions/709621


### Manage GPU hours
https://udacity.zendesk.com/hc/en-us/articles/360014002451-How-do-I-manage-my-GPU-hours-If-applicable-


### Workspace Idel Time
**Desktop will disconnect after 30 minutes of inactivity.**
https://udacity.zendesk.com/hc/en-us/articles/360019820872-How-to-keep-my-workspace-from-going-idle-

https://github.com/udacity/workspaces-student-support/tree/master/jupyter


### Class distribution
https://knowledge.udacity.com/questions/729875


### Firefox crush
https://knowledge.udacity.com/questions/723823
sudo apt-get update
sudo apt-get install chromium-browser
sudo chromium-browser --no-sandbox 

I would recommend to update to the latest firefox version using
sudo apt-get update
sudo apt-get install firefox



## Submission Template

### Project overview
This section should contain a brief description of the project and what we are trying to achieve. Why is object detection such an important component of self driving car systems?

### Set up
This section should contain a brief description of the steps to follow to run the code for this repository.

### Dataset
#### Dataset analysis
This section should contain a quantitative and qualitative description of the dataset. It should include images, charts and other visualizations.
#### Cross validation
This section should detail the cross validation strategy and justify your approach.

### Training
#### Reference experiment
This section should detail the results of the reference experiment. It should includes training metrics and a detailed explanation of the algorithm's performances.

#### Improve on the reference
This section should highlight the different strategies you adopted to improve your model. It should contain relevant figures and details of your findings.


In [44]:
import glob
import shutil
import numpy as np

In [18]:
files = np.array(glob.glob('./data/waymo/training_and_validation/*.tfrecord'))
files

array(['./data/waymo/training_and_validation/segment-10599748131695282446_1380_000_1400_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10927752430968246422_4940_000_4960_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-11219370372259322863_5320_000_5340_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-11252086830380107152_1540_000_1560_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-11489533038039664633_4820_000_4840_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10770759614217273359_1465_000_1485_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10206293520369375008_2796_800_2816_800_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10391312872392849784_4099_400_4119_400_with_camera_labels.tfrecord',
       './data/waymo/tra

In [29]:
files.shape

(97,)

In [22]:
np.random.shuffle(files)

In [25]:
files

array(['./data/waymo/training_and_validation/segment-11674150664140226235_680_000_700_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10517728057304349900_3360_000_3380_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10498013744573185290_1240_000_1260_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10750135302241325253_180_000_200_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-11588853832866011756_2184_462_2204_462_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-11126313430116606120_1439_990_1459_990_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10526338824408452410_5714_660_5734_660_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10241508783381919015_2889_360_2909_360_with_camera_labels.tfrecord',
       './data/waymo/trainin

In [40]:
train_ratio = 0.8
val_start_idx = int(train_ratio * files.shape[0])

In [41]:
train_files = files[0: val_start_idx]
train_files

array(['./data/waymo/training_and_validation/segment-11674150664140226235_680_000_700_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10517728057304349900_3360_000_3380_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10498013744573185290_1240_000_1260_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10750135302241325253_180_000_200_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-11588853832866011756_2184_462_2204_462_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-11126313430116606120_1439_990_1459_990_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10526338824408452410_5714_660_5734_660_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10241508783381919015_2889_360_2909_360_with_camera_labels.tfrecord',
       './data/waymo/trainin

In [42]:
val_files = files[val_start_idx:]
val_files

array(['./data/waymo/training_and_validation/segment-10724020115992582208_7660_400_7680_400_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-11060291335850384275_3761_210_3781_210_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10094743350625019937_3420_000_3440_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10226164909075980558_180_000_200_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-10664823084372323928_4360_000_4380_000_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-11343624116265195592_5910_530_5930_530_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-11623618970700582562_2840_367_2860_367_with_camera_labels.tfrecord',
       './data/waymo/training_and_validation/segment-11566385337103696871_5740_000_5760_000_with_camera_labels.tfrecord',
       './data/waymo/train

In [43]:
val_files.shape

(20,)

In [52]:
shutil.move('./data/waymo/training_and_validation/test.tfrecord', './data/waymo/train/')

FileNotFoundError: [Errno 2] No such file or directory: './data/waymo/training_and_validation/test.tfrecord'

In [54]:
for file in train_files:
#     print(file)
    shutil.move(file, './data/waymo/train/')

In [55]:
for file in val_files:
#     print(file)
    shutil.move(file, './data/waymo/val/')